In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install PyWavelets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 31.1 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Concatenate, Dropout, Input, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import os
import gc

# Path configurations
DATA_PATH = {
   'train_images': '/content/drive/MyDrive/Final Year Project Material/test',
    'test_images': '/content/drive/MyDrive/Final Year Project Material/train',
    'train_haar': '/content/drive/MyDrive/Final Year Project Material/NUUA_HAAR TRANSFORMED/haar_transformed_data_64x64-project-test.csv',
    'test_haar': '/content/drive/MyDrive/Final Year Project Material/NUUA_HAAR TRANSFORMED/haar_transformed_data_64x64-project-train.csv'
}

# Constants
IMG_SIZE = (299, 299)  # InceptionV3 requires 299x299 input
BATCH_SIZE = 32
NUM_CLASSES = 2
NUM_EPOCHS = 50
VERBOSE = 1

def setup_data_generators():
    datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        preprocessing_function=tf.keras.applications.inception_v3.preprocess_input
    )

    train_generator = datagen.flow_from_directory(
        DATA_PATH['train_images'],
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=True
    )

    test_datagen = ImageDataGenerator(
        rescale=1./255,
        preprocessing_function=tf.keras.applications.inception_v3.preprocess_input
    )

    test_generator = test_datagen.flow_from_directory(
        DATA_PATH['test_images'],
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False
    )

    return train_generator, test_generator


def load_haar_features():
    train_haar_df = pd.read_csv(DATA_PATH['train_haar'])
    test_haar_df = pd.read_csv(DATA_PATH['test_haar'])
    return train_haar_df, test_haar_df


def create_modified_inception_with_gradual_unfreezing():
    base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(*IMG_SIZE, 3))

    for layer in base_model.layers:
        layer.trainable = False

    x = Flatten()(base_model.output)
    x = Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
    x = Dropout(0.5)(x)
    model = Model(inputs=base_model.input, outputs=x)

    return base_model, model


def create_fusion_model(cnn_features_shape, haar_features_shape):
    cnn_input = Input(shape=cnn_features_shape)
    cnn_dense = Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(cnn_input)
    haar_input = Input(shape=haar_features_shape)
    haar_dense = Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(haar_input)

    fusion = Concatenate()([cnn_dense, haar_dense])
    x = Dense(256, activation='relu')(fusion)
    x = Dropout(0.5)(x)
    output = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=[cnn_input, haar_input], outputs=output)
    return model

def extract_cnn_features(model, generator):
    features, labels = [], []
    steps = len(generator)
    for i in range(steps):
        batch_x, batch_y = generator[i]
        batch_features = model.predict(batch_x, verbose=0)
        features.append(batch_features)
        labels.append(batch_y)
        if (i + 1) % 2 == 0:
            print(f"Processed {i + 1}/{steps} batches")
    return np.vstack(features), np.concatenate(labels)

def prepare_haar_features(haar_df):
    X = haar_df.drop('label', axis=1).values
    y = haar_df['label'].values
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    return X_scaled, y

def gradual_unfreeze_and_train(base_model, fusion_model, train_cnn_features, train_haar_features,
                                train_labels, test_cnn_features, test_haar_features, test_labels):
    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-6,
        verbose=1
    )

    early_stopping = EarlyStopping(
        monitor='val_accuracy',
        patience=10,
        restore_best_weights=True
    )

    # InceptionV3 has 311 layers compared to VGG16's 19 layers
    # Adjust unfreezing strategy for InceptionV3
    unfreeze_stages = [
        (base_model.layers[-30:], 5e-5),        # Last 30 layers
        (base_model.layers[-60:-30], 1e-5),     # Next 30 layers
        (base_model.layers[-90:-60], 5e-6)      # Another 30 layers
    ]

    best_accuracy = 0

    for stage, (layers_to_unfreeze, learning_rate) in enumerate(unfreeze_stages, 1):
        for layer in layers_to_unfreeze:
            layer.trainable = True

        optimizer = Adam(learning_rate=learning_rate)
        fusion_model.compile(
            optimizer=optimizer,
            loss='binary_crossentropy',
            metrics=['accuracy']
        )

        history = fusion_model.fit(
            [train_cnn_features, train_haar_features],
            train_labels,
            validation_data=([test_cnn_features, test_haar_features], test_labels),
            callbacks=[reduce_lr, early_stopping],
            epochs=NUM_EPOCHS,
            batch_size=BATCH_SIZE,
            verbose=VERBOSE
        )

        test_predictions = fusion_model.predict([test_cnn_features, test_haar_features])
        predictions = (test_predictions > 0.5).astype(int).flatten()
        current_accuracy = accuracy_score(test_labels, predictions)

        print(f"\nStage {stage} Accuracy: {current_accuracy:.4f}")

        if current_accuracy > best_accuracy:
            best_accuracy = current_accuracy
            # First Save
            fusion_model.save(f'/content/drive/MyDrive/Final Year Project Material/Suraj Code Copy/Best Fusion Model/InceptionV3_50_Epochs_64x64_HAAR_Stepwise_Unfreezing/best_fusion_model_stage_{stage}.h5')
            # Second Save
            fusion_model.save(f'/content/drive/MyDrive/Final Year Project Material/Save Models/Best Fusion Model/InceptionV3_50_Epochs_64x64_HAAR_Stepwise_Unfreezing/best_fusion_model_stage_{stage}.h5')

    return fusion_model, best_accuracy

def evaluate_model(model, test_cnn_features, test_haar_features, test_labels):
    predictions = model.predict([test_cnn_features, test_haar_features])
    predictions = (predictions > 0.5).astype(int).flatten()

    acc = accuracy_score(test_labels, predictions)
    conf_matrix = confusion_matrix(test_labels, predictions)
    class_report = classification_report(test_labels, predictions)

    print(f'Accuracy: {acc:.4f}')
    print('\nConfusion Matrix:\n', conf_matrix)
    print('\nClassification Report:\n', class_report)

    return acc, conf_matrix, class_report

def train_fusion_model():
    print("Loading data...")
    train_generator, test_generator = setup_data_generators()
    train_haar_df, test_haar_df = load_haar_features()

    print("\nCreating and training models...")
    base_model, feature_extractor = create_modified_inception_with_gradual_unfreezing()

    print("\nExtracting CNN features from training data...")
    train_cnn_features, train_labels = extract_cnn_features(feature_extractor, train_generator)

    print("\nExtracting CNN features from testing data...")
    test_cnn_features, test_labels = extract_cnn_features(feature_extractor, test_generator)

    print("\nPreparing HAAR features...")
    train_haar_features, _ = prepare_haar_features(train_haar_df)
    test_haar_features, _ = prepare_haar_features(test_haar_df)

    print("\nFeature Shapes:")
    print("CNN Features Shape:", train_cnn_features.shape)
    print("HAAR Features Shape:", train_haar_features.shape)

    fusion_model = create_fusion_model(train_cnn_features.shape[1:], train_haar_features.shape[1:])

    final_model, best_accuracy = gradual_unfreeze_and_train(
        base_model,
        fusion_model,
        train_cnn_features,
        train_haar_features,
        train_labels,
        test_cnn_features,
        test_haar_features,
        test_labels
    )

    print(f"\nBest Model Accuracy: {best_accuracy:.4f}")

    return final_model, None, feature_extractor

def main():
    tf.keras.backend.clear_session()
    gc.collect()

    for path_name, path in DATA_PATH.items():
        if not os.path.exists(path):
            raise FileNotFoundError(f"Path not found: {path_name} - {path}")

    fusion_model, history, feature_extractor = train_fusion_model()

    print("\nSaving models...")

    # First Save
    feature_extractor.save('/content/drive/MyDrive/Final Year Project Material/Suraj Code Copy/InceptionV3_50_Epochs_64x64_HAAR_Stepwise_Unfreezing/feature_extractor_InceptionV3_50_epochs_64x64_haar_stepwise_unfreezing.h5')
    fusion_model.save('/content/drive/MyDrive/Final Year Project Material/Suraj Code Copy/InceptionV3_50_Epochs_64x64_HAAR_Stepwise_Unfreezing/fusion_model_InceptionV3_50_epochs_64x64_haar_stepwise_unfreezing.h5')

    # Second Save
    feature_extractor.save('/content/drive/MyDrive/Final Year Project Material/Save Models/InceptionV3_50_Epochs_64x64_HAAR_Stepwise_Unfreezing/feature_extractor_InceptionV3_50_epochs_64x64_haar_stepwise_unfreezing.h5')
    fusion_model.save('/content/drive/MyDrive/Final Year Project Material/Save Models/InceptionV3_50_Epochs_64x64_HAAR_Stepwise_Unfreezing/fusion_model_InceptionV3_50_epochs_64x64_haar_stepwise_unfreezing.h5')

    return fusion_model, history, feature_extractor

if __name__ == "__main__":
    fusion_model, history, feature_extractor = main()

Loading data...
Found 9123 images belonging to 2 classes.
Found 3491 images belonging to 2 classes.

Creating and training models...
87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step

Extracting CNN features from training data...
Processed 2/286 batches
Processed 4/286 batches
Processed 6/286 batches
Processed 8/286 batches
Processed 10/286 batches
Processed 12/286 batches
Processed 14/286 batches
Processed 16/286 batches
Processed 18/286 batches
Processed 20/286 batches
Processed 22/286 batches
Processed 24/286 batches
Processed 26/286 batches
Processed 28/286 batches
Processed 30/286 batches
Processed 32/286 batches
Processed 34/286 batches
Processed 36/286 batches
Processed 38/286 batches
Processed 40/286 batches
Processed 42/286 batches
Processed 44/286 batches
Processed 46/286 batches
Processed 48/286 batches
Processed 50/286 batches
Processed 52/286 batches
Processed 54/286 batches
Processed 56/286 batches
Processed 58/286 batches
Processed 60/286 batches
Processed 62/286 batche


Stage 1 Accuracy: 0.9304


Epoch 1/50
286/286 ━━━━━━━━━━━━━━━━━━━━ 11s 31ms/step - accuracy: 0.9632 - loss: 0.5996 - val_accuracy: 0.9063 - val_loss: 0.7545 - learning_rate: 1.0000e-05
Epoch 2/50
286/286 ━━━━━━━━━━━━━━━━━━━━ 7s 26ms/step - accuracy: 0.9661 - loss: 0.5912 - val_accuracy: 0.9043 - val_loss: 0.7619 - learning_rate: 1.0000e-05
Epoch 3/50
286/286 ━━━━━━━━━━━━━━━━━━━━ 8s 29ms/step - accuracy: 0.9639 - loss: 0.5870 - val_accuracy: 0.9055 - val_loss: 0.7653 - learning_rate: 1.0000e-05
Epoch 4/50
286/286 ━━━━━━━━━━━━━━━━━━━━ 9s 31ms/step - accuracy: 0.9669 - loss: 0.5766 - val_accuracy: 0.9112 - val_loss: 0.7374 - learning_rate: 1.0000e-05
Epoch 5/50
286/286 ━━━━━━━━━━━━━━━━━━━━ 9s 26ms/step - accuracy: 0.9707 - loss: 0.5742 - val_accuracy: 0.9072 - val_loss: 0.7459 - learning_rate: 1.0000e-05
Epoch 6/50
286/286 ━━━━━━━━━━━━━━━━━━━━ 11s 30ms/step - accuracy: 0.9698 - loss: 0.5701 - val_accuracy: 0.9109 - val_loss: 0.7313 - learning_rate: 1.0000e-05
Epoch 7/50
286/286 ━━━━━━━━━━━━━━━━━━━━ 8s 28ms/step - a


Stage 3 Accuracy: 0.9123

Best Model Accuracy: 0.9304

Saving models...
